In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os

from tqdm.notebook import tqdm

from particle import Particle # https://github.com/scikit-hep/particle

import pickle

import json

import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [2]:
# reco BEE link: https://www.phy.bnl.gov/twister/bee/set/f51043f3-10d4-49b3-9866-bc7fbcf5d4fe/event/list/
# truth BEE link: https://www.phy.bnl.gov/twister/bee/set/7ad1b4aa-a181-435a-9117-d37c0c9ca677/event/list/


In [3]:
# loading truth spacepoints from each event from the BEE link json (couldn't find this in the saved celltree.root output, maybe "mc_trackPosition", but I wasn't able to load it with uproot)

truth_spacepoints = {}
for dir in tqdm(os.listdir("input_files/misclustering_candidate_truth_outputs/bee/data/")):
    json_file = f"input_files/misclustering_candidate_truth_outputs/bee/data/{dir}/{dir}-truthDepo.json"
    with open(json_file, "r") as f:
        truth_data = json.load(f)
    run = int(truth_data["runNo"])
    subrun = int(truth_data["subRunNo"])
    event = int(truth_data["eventNo"])
    curr_truth_spacepoints = np.column_stack((np.array(truth_data["x"]) + 2, np.array(truth_data["y"]), np.array(truth_data["z"])))
    truth_spacepoints[(run, subrun, event)] = curr_truth_spacepoints


  0%|          | 0/83 [00:00<?, ?it/s]

In [4]:
# loading reco spacepoints from each event

reco_nu_spacepoints = {}
reco_cosmic_spacepoints = {}
reco_nu_vtxs = {}
for file in tqdm(os.listdir("input_files/misclustering_candidate_nue_files")):
    if not file.endswith(".root"):
        continue

    f = uproot.open(f"input_files/misclustering_candidate_nue_files/{file}")

    rse_df = f["Trun"].arrays(["runNo", "subRunNo", "eventNo"], library="pd")

    run = int(rse_df["runNo"].iloc[0])
    subrun = int(rse_df["subRunNo"].iloc[0])
    event = int(rse_df["eventNo"].iloc[0])

    rse = (run, subrun, event)

    curr_reco_nu_spacepoints_dic = f["T_rec_charge_blob"].arrays(["x", "y", "z", "q"], library="np")
    curr_reco_nu_spacepoints = np.column_stack((curr_reco_nu_spacepoints_dic["x"], curr_reco_nu_spacepoints_dic["y"], curr_reco_nu_spacepoints_dic["z"], curr_reco_nu_spacepoints_dic["q"]))

    curr_cosmic_spacepoints_dic = f["T_cluster"].arrays(["x", "y", "z", "q"], library="np")
    curr_cosmic_spacepoints = np.column_stack((curr_cosmic_spacepoints_dic["x"], curr_cosmic_spacepoints_dic["y"], curr_cosmic_spacepoints_dic["z"], curr_cosmic_spacepoints_dic["q"]))

    reco_nu_spacepoints[rse] = curr_reco_nu_spacepoints
    reco_cosmic_spacepoints[rse] = curr_cosmic_spacepoints

    reco_nu_vtx_dic = f["T_vtx"].arrays(["x", "y", "z", "flag_main"], library="np")
    reco_nu_vtx_x = reco_nu_vtx_dic["x"][reco_nu_vtx_dic["flag_main"] == 1][0]
    reco_nu_vtx_y = reco_nu_vtx_dic["y"][reco_nu_vtx_dic["flag_main"] == 1][0]
    reco_nu_vtx_z = reco_nu_vtx_dic["z"][reco_nu_vtx_dic["flag_main"] == 1][0]

    reco_nu_vtxs[rse] = (reco_nu_vtx_x, reco_nu_vtx_y, reco_nu_vtx_z)


  0%|          | 0/85 [00:00<?, ?it/s]

In [5]:
# downsampling the cosmic spacepoints

from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

def fps_sampling(points, n_samples):
    """
    Perform an optimized Farthest Point Sampling (FPS) using NumPy.

    :param points: numpy array of shape (N, 3) representing the point cloud
    :param n_samples: number of points to sample
    :return: indices of sampled points
    """
    N = points.shape[0]
    sampled_indices = np.zeros(n_samples, dtype=int)
    distances = np.full(N, np.inf)

    # Choose the first point randomly
    sampled_indices[0] = np.random.randint(N)
    
    # Efficiently compute distances using vectorized operations
    for i in range(1, n_samples):
        # Update minimum distances
        diff = points - points[sampled_indices[i - 1]]
        new_distances = np.einsum('ij,ij->i', diff, diff)  # Faster squared Euclidean distance
        distances = np.minimum(distances, new_distances)
        
        # Select the point farthest from the existing sampled set
        sampled_indices[i] = np.argmax(distances)

    sampled_points = points[sampled_indices]
    
    return sampled_points

def fps_clustering_downsample(points, n_samples, debug=False):
    """
    Downsample the point cloud using FPS and clustering.
    
    :param points: numpy array of shape (N, 3) representing the point cloud
    :param n_samples: number of points in the downsampled cloud
    :return: downsampled point cloud
    """

    if debug:
        print(f"downsampling {points.shape[0]} points to {n_samples} points")

    if debug:
        print(f"performing FPS...", end="", flush=True)

    # Perform FPS to get initial samples
    sampled_points = fps_sampling(points, n_samples)
    
    if debug:
        print(f"done", flush=True)

    if debug:
        print(f"performing KMeans...", end="", flush=True)

    # Use K-means clustering to associate other points with the samples
    kmeans = KMeans(n_clusters=n_samples, init=sampled_points, n_init=1)
    kmeans.fit(points)

    if debug:
        print(f"done", flush=True)
        
    return kmeans.cluster_centers_

def get_min_dists(points_A, points_B):
    """
    Get the minimum distance between each point in points_A and all the points in points_B.
    """
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(points_B)
    distances, _ = nbrs.kneighbors(points_A)
    return distances


In [6]:
recalculate_downsampling = False
if recalculate_downsampling:
    downsampled_cosmic_spacepoints = {}
    for rse in tqdm(reco_cosmic_spacepoints.keys()):
        downsampled_cosmic_spacepoints[rse] = fps_clustering_downsample(reco_cosmic_spacepoints[rse][:, :3], 1000)
    with open("input_files/misclustering_candidate_nue_files/downsampled_cosmic_spacepoints.pkl", "wb") as f:
        pickle.dump(downsampled_cosmic_spacepoints, f)
else:
    with open("input_files/misclustering_candidate_nue_files/downsampled_cosmic_spacepoints.pkl", "rb") as f:
        downsampled_cosmic_spacepoints = pickle.load(f)


  0%|          | 0/83 [00:00<?, ?it/s]

In [18]:
# splitting based on distances to things

real_nu_reco_nu_downsampled_spacepoints = {}
real_nu_reco_cosmic_downsampled_spacepoints = {}
real_cosmic_reco_nu_downsampled_spacepoints = {}
real_cosmic_reco_cosmic_downsampled_spacepoints = {}
real_cosmic_reco_nu_downsampled_spacepoints = {}
far_from_vtx_downsampled_spacepoints = {}

for rse in tqdm(downsampled_cosmic_spacepoints.keys()):

    min_cosmic_truth_dists = get_min_dists(downsampled_cosmic_spacepoints[rse][:, :3], truth_spacepoints[rse][:, :3])
    min_cosmic_reco_nu_dists = get_min_dists(downsampled_cosmic_spacepoints[rse][:, :3], reco_nu_spacepoints[rse][:, :3])
    cosmic_vtx_dists = np.sqrt((downsampled_cosmic_spacepoints[rse][:, 0] - reco_nu_vtxs[rse][0])**2
                                + (downsampled_cosmic_spacepoints[rse][:, 1] - reco_nu_vtxs[rse][1])**2
                                  + (downsampled_cosmic_spacepoints[rse][:, 2] - reco_nu_vtxs[rse][2])**2)

    not_far_from_vtx_indices = np.where(cosmic_vtx_dists < 200.0)[0]
    far_from_vtx_indices = np.where(cosmic_vtx_dists >= 200.0)[0]

    close_to_truth_indices = np.where(min_cosmic_truth_dists < 5.0)[0]
    far_from_truth_indices = np.where(min_cosmic_truth_dists >= 5.0)[0]

    close_to_reco_nu_indices = np.where(min_cosmic_reco_nu_dists < 5.0)[0]
    far_from_reco_nu_indices = np.where(min_cosmic_reco_nu_dists >= 5.0)[0]


    real_nu_reco_nu_indices = np.intersect1d(close_to_reco_nu_indices, close_to_truth_indices)
    real_nu_reco_cosmic_indices = np.intersect1d(far_from_reco_nu_indices, close_to_truth_indices)
    real_cosmic_reco_nu_indices = np.intersect1d(close_to_reco_nu_indices, far_from_truth_indices)
    real_cosmic_reco_cosmic_indices = np.intersect1d(far_from_reco_nu_indices, far_from_truth_indices)

    real_nu_reco_nu_indices = np.intersect1d(real_nu_reco_nu_indices, not_far_from_vtx_indices)
    real_nu_reco_cosmic_indices = np.intersect1d(real_nu_reco_cosmic_indices, not_far_from_vtx_indices)
    real_cosmic_reco_nu_indices = np.intersect1d(real_cosmic_reco_nu_indices, not_far_from_vtx_indices)
    real_cosmic_reco_cosmic_indices = np.intersect1d(real_cosmic_reco_cosmic_indices, not_far_from_vtx_indices)

    missing_nu_indices = np.intersect1d(close_to_truth_indices, far_from_reco_nu_indices)
    missing_nu_indices = np.intersect1d(missing_nu_indices, not_far_from_vtx_indices)
    cosmic_indices = np.intersect1d(far_from_truth_indices, far_from_reco_nu_indices)
    cosmic_indices = np.intersect1d(cosmic_indices, not_far_from_vtx_indices)
    nu_indices = close_to_reco_nu_indices
    nu_indices = np.intersect1d(close_to_reco_nu_indices, not_far_from_vtx_indices)

    real_nu_reco_cosmic_downsampled_spacepoints[rse] = downsampled_cosmic_spacepoints[rse][real_nu_reco_cosmic_indices, :]
    real_nu_reco_nu_downsampled_spacepoints[rse] = downsampled_cosmic_spacepoints[rse][real_nu_reco_nu_indices, :]
    real_cosmic_reco_nu_downsampled_spacepoints[rse] = downsampled_cosmic_spacepoints[rse][real_cosmic_reco_nu_indices, :]
    real_cosmic_reco_cosmic_downsampled_spacepoints[rse] = downsampled_cosmic_spacepoints[rse][real_cosmic_reco_cosmic_indices, :]
    far_from_vtx_downsampled_spacepoints[rse] = downsampled_cosmic_spacepoints[rse][far_from_vtx_indices, :]

    if rse == (13779, 223, 11160):
        print(f"{rse} spacepoint counts:")
        print(f"num cosmic spacepoints: {len(reco_cosmic_spacepoints[rse])}")
        print(f"num downsampled cosmic spacepoints: {len(downsampled_cosmic_spacepoints[rse])}")
        print(f"num truth spacepoints: {len(truth_spacepoints[rse])}")
        print(f"num reco neutrino spacepoints: {len(reco_nu_spacepoints[rse])}")
        print(f"num nearby reco cosmic downsampled cosmic spacepoints: {len(real_nu_reco_cosmic_downsampled_spacepoints[rse]) + len(real_cosmic_reco_cosmic_downsampled_spacepoints[rse])}")
        print(f"num nearby cosmic downsampled spacepoints: {len(not_far_from_vtx_indices)}")

  0%|          | 0/83 [00:00<?, ?it/s]

(13779, 223, 11160) spacepoint counts:
num cosmic spacepoints: 131889
num downsampled cosmic spacepoints: 1000
num truth spacepoints: 8338
num reco neutrino spacepoints: 266
num nearby reco cosmic downsampled cosmic spacepoints: 146
num nearby cosmic downsampled spacepoints: 189


In [8]:
print(8*3*1_000_000*131_889/1e12)
print(8*3*1_000_000*8_338/1e12)
print(8*3*1_000_000*266/1e12)

3.165336
0.200112
0.006384


In [9]:
# making detector boundary points

tpc_min_x = -1.
tpc_max_x = 254.3
tpc_min_y = -115.
tpc_max_y = 117.
tpc_min_z = 0.6
tpc_max_z = 1036.4

def generate_box_edge_points(x_min, x_max, y_min, y_max, z_min, z_max, num_points_per_edge=10):

    # Generate points along edges parallel to X-axis
    t = np.linspace(0, 1, num_points_per_edge)
    x_edges = [
        np.column_stack([
            x_min + t * (x_max - x_min),
            np.full_like(t, y_min),
            np.full_like(t, z_min)
        ]),
        np.column_stack([
            x_min + t * (x_max - x_min),
            np.full_like(t, y_max),
            np.full_like(t, z_min)
        ]),
        np.column_stack([
            x_min + t * (x_max - x_min),
            np.full_like(t, y_min),
            np.full_like(t, z_max)
        ]),
        np.column_stack([
            x_min + t * (x_max - x_min),
            np.full_like(t, y_max),
            np.full_like(t, z_max)
        ])
    ]
    
    # Generate points along edges parallel to Y-axis
    y_edges = [
        np.column_stack([
            np.full_like(t, x_min),
            y_min + t * (y_max - y_min),
            np.full_like(t, z_min)
        ]),
        np.column_stack([
            np.full_like(t, x_max),
            y_min + t * (y_max - y_min),
            np.full_like(t, z_min)
        ]),
        np.column_stack([
            np.full_like(t, x_min),
            y_min + t * (y_max - y_min),
            np.full_like(t, z_max)
        ]),
        np.column_stack([
            np.full_like(t, x_max),
            y_min + t * (y_max - y_min),
            np.full_like(t, z_max)
        ])
    ]
    
    # Generate points along edges parallel to Z-axis
    z_edges = [
        np.column_stack([
            np.full_like(t, x_min),
            np.full_like(t, y_min),
            z_min + t * (z_max - z_min)
        ]),
        np.column_stack([
            np.full_like(t, x_max),
            np.full_like(t, y_min),
            z_min + t * (z_max - z_min)
        ]),
        np.column_stack([
            np.full_like(t, x_min),
            np.full_like(t, y_max),
            z_min + t * (z_max - z_min)
        ]),
        np.column_stack([
            np.full_like(t, x_max),
            np.full_like(t, y_max),
            z_min + t * (z_max - z_min)
        ])
    ]
    
    # Combine all edges
    all_points = np.vstack(x_edges + y_edges + z_edges)
    return all_points


detector_boundary_points = generate_box_edge_points(tpc_min_x, tpc_max_x, tpc_min_y, tpc_max_y, tpc_min_z, tpc_max_z, num_points_per_edge=100)
x_width = tpc_max_x - tpc_min_x
expanded_detector_boundary_points = generate_box_edge_points(tpc_min_x - x_width, tpc_max_x + x_width, tpc_min_y, tpc_max_y, tpc_min_z, tpc_max_z, num_points_per_edge=100)


In [10]:
rse_list = list(reco_nu_spacepoints.keys())

In [16]:
index = 50

# 10 is good
# 50 is perfect
# 51 is good
# 54 is good
# 55 is okay
# 57 is good, with a true cosmic reco nu track

rse = rse_list[index]

print("rse:", rse)

curr_detector_boundary_points = detector_boundary_points
curr_expanded_detector_boundary_points = expanded_detector_boundary_points

curr_reco_nu_spacepoints = reco_nu_spacepoints[rse]
curr_truth_spacepoints = truth_spacepoints[rse]

curr_far_from_vtx_downsampled_spacepoints = far_from_vtx_downsampled_spacepoints[rse]

curr_real_nu_reco_cosmic_downsampled_spacepoints = real_nu_reco_cosmic_downsampled_spacepoints[rse]
curr_real_nu_reco_nu_downsampled_spacepoints = real_nu_reco_nu_downsampled_spacepoints[rse]
curr_real_cosmic_reco_nu_downsampled_spacepoints = real_cosmic_reco_nu_downsampled_spacepoints[rse]
curr_real_cosmic_reco_cosmic_downsampled_spacepoints = real_cosmic_reco_cosmic_downsampled_spacepoints[rse]

curr_downsampled_cosmic_spacepoints = downsampled_cosmic_spacepoints[rse]

fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scene'}]])

fig.add_trace(go.Scatter3d(
    x=curr_expanded_detector_boundary_points[:, 2],
    y=curr_expanded_detector_boundary_points[:, 0],
    z=curr_expanded_detector_boundary_points[:, 1],
    mode='markers',
    marker=dict(
        size=0.2,
        color='black',
        opacity=0.8
    ),
    name='Expanded TPC Boundary'
))

fig.add_trace(go.Scatter3d(
    x=curr_detector_boundary_points[:, 2],
    y=curr_detector_boundary_points[:, 0],
    z=curr_detector_boundary_points[:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color='black',
        opacity=0.8
    ),
    name='TPC Boundary'
))

fig.add_trace(go.Scatter3d(
    x=[reco_nu_vtxs[rse][2]],
    y=[reco_nu_vtxs[rse][0]],
    z=[reco_nu_vtxs[rse][1]],
    mode='markers',
    marker=dict(size=10, color='orange', opacity=1),
    name='Reco Neutrino Vertex'
))

fig.add_trace(go.Scatter3d(
    x=curr_real_cosmic_reco_cosmic_downsampled_spacepoints[:, 2],
    y=curr_real_cosmic_reco_cosmic_downsampled_spacepoints[:, 0],
    z=curr_real_cosmic_reco_cosmic_downsampled_spacepoints[:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='blue',
        opacity=0.8
    ),
    name='Correctly Identified Cosmic Downsampled Spacepoints'
))

fig.add_trace(go.Scatter3d(
    x=curr_real_nu_reco_nu_downsampled_spacepoints[:, 2],
    y=curr_real_nu_reco_nu_downsampled_spacepoints[:, 0],
    z=curr_real_nu_reco_nu_downsampled_spacepoints[:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='green',
        opacity=0.8
    ),
    name='Found Neutrino Downsampled Spacepoints'
))

fig.add_trace(go.Scatter3d(
    x=curr_real_nu_reco_cosmic_downsampled_spacepoints[:, 2],
    y=curr_real_nu_reco_cosmic_downsampled_spacepoints[:, 0],
    z=curr_real_nu_reco_cosmic_downsampled_spacepoints[:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='red',
        opacity=0.8
    ),
    name='Misidentified Neutrino Downsampled Spacepoints'
))

fig.add_trace(go.Scatter3d(
    x=curr_real_cosmic_reco_nu_downsampled_spacepoints[:, 2],
    y=curr_real_cosmic_reco_nu_downsampled_spacepoints[:, 0],
    z=curr_real_cosmic_reco_nu_downsampled_spacepoints[:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='brown',
        opacity=0.8
    ),
    name='Misidentified Cosmic Downsampled Spacepoints'
))

fig.add_trace(go.Scatter3d(
    x=np.append(curr_real_nu_reco_cosmic_downsampled_spacepoints[:, 2], curr_real_cosmic_reco_cosmic_downsampled_spacepoints[:, 2]),
    y=np.append(curr_real_nu_reco_cosmic_downsampled_spacepoints[:, 0], curr_real_cosmic_reco_cosmic_downsampled_spacepoints[:, 0]),
    z=np.append(curr_real_nu_reco_cosmic_downsampled_spacepoints[:, 1], curr_real_cosmic_reco_cosmic_downsampled_spacepoints[:, 1]),
    mode='markers',
    marker=dict(
        size=3,
        color='black',
        opacity=0.8
    ),
    name='All Reconstructed Cosmic Downsampled Spacepoints'
))

fig.add_trace(go.Scatter3d(
    x=curr_downsampled_cosmic_spacepoints[:, 2],
    y=curr_downsampled_cosmic_spacepoints[:, 0],
    z=curr_downsampled_cosmic_spacepoints[:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='black',
        opacity=0.8
    ),
    name='All Cosmic Downsampled Spacepoints'
))



fig.add_trace(go.Scatter3d(
    x=curr_reco_nu_spacepoints[:, 2],
    y=curr_reco_nu_spacepoints[:, 0],
    z=curr_reco_nu_spacepoints[:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color=curr_reco_nu_spacepoints[:, 3],
        colorscale='Jet',
        opacity=0.8
    ),
    name='Neutrino Cluster Trajectory Spacepoints',
    visible='legendonly'
))

fig.add_trace(go.Scatter3d(
    x=curr_truth_spacepoints[:, 2],
    y=curr_truth_spacepoints[:, 0],
    z=curr_truth_spacepoints[:, 1],
    mode='markers',
    marker=dict(size=2, color='orange', opacity=0.8),
    name='BEE Truth Spacepoints',
    visible='legendonly'
))



fig.update_layout(
    scene=dict(
        xaxis_title='z',
        yaxis_title='x',
        zaxis_title='y',
        aspectratio=dict(
            x=5,
            y=3,
            z=1
        ),
    ),
    width=2000,
    height=1200,
    autosize=False,
    scene_camera=dict(
        eye=dict(x=-1.5, y=-1.5, z=1.5)
    )
)

fig.show(renderer="browser")


rse: (13779, 223, 11160)
